# Setup and Test CoT, CoVe and Self-Refine

## Helper methods

### Extract multiple-choice answer from response

In [387]:
import re

def extract_answer_letter(response):
    # Match "Resposta final: C)" or "Resposta final: C"
    match = re.search(r"resposta final\s*[:\-]?\s*([A-E])\s*\)?", response, re.IGNORECASE | re.DOTALL)
    if not match:
        # Try fallback patterns
        match = re.search(r"letra\s+([A-E])\b", response, re.IGNORECASE)
    return match.group(1).upper() if match else None

### Calculate overall accuracy of answers

In [388]:
def calculate_accuracy(results):
    """Returns overall accuracy and count of None predictions from a list of result dicts."""
    total_answered = sum(1 for r in results if r["predicted"] is not None)
    correct = sum(r["correct"] for r in results if r["predicted"] is not None)
    total = len(results)
    none_count = total - total_answered
    accuracy = (correct / total_answered) * 100 if total_answered > 0 else 0
    return correct, total_answered, accuracy, none_count

### Group results by subject

In [389]:
import pandas as pd

def results_by_subject(results):
    """Aggregates accuracy grouped by subject"""
    df = pd.DataFrame(results)
    if "subject" not in df.columns:
        print("⚠️ 'subject' not found in results.")
        return None
    
    summary = df.groupby("subject")["correct"].agg(["sum", "count"])
    summary["accuracy (%)"] = (summary["sum"] / summary["count"]) * 100
    return summary

### Save results to csv file

In [390]:
from datetime import datetime
import os

def save_results_csv(df, method_name):
    """
    Save a DataFrame as a CSV file in a 'results/<method_name>' subfolder with a timestamped filename.

    Parameters:
    - df: pandas DataFrame to save
    - method_name: e.g., 'cot', 'cov', 'self-refine'

    Returns:
    - The full filename used
    """
    # Define target folder and create it if needed
    folder = os.path.join("results", method_name)
    os.makedirs(folder, exist_ok=True)

    # Create timestamped filename
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = os.path.join(folder, f"{method_name}_results_{timestamp}.csv")

    # Save the file
    df.to_csv(filename, index=False)
    print(f"✅ Results saved to {filename}")
    return filename

## Chain-of-Thought (CoT) template

In [391]:
def build_cot_prompt(question_obj):
    
    few_shot_examples = """Você verá abaixo alguns exemplos de como a pergunta deve ser respondida passo a passo. Leia atentamente os exemplos e, em seguida, responda a pergunta que vem depois deles.
    
        ### Exemplo 1:
        
        Pergunta:
        Urgência emocional. Se tudo é para ontem, se a vida engata uma primeira e sai em
        disparada, se não há mais tempo para paradas estratégicas, caímos fatalmente no vício de querer
        que os amores sejam igualmente resolvidos num átimo de segundo. Temos pressa para ouvir “eu
        te amo”. Não vemos a hora de que fiquem estabelecidas as regras de convívio: somos namorados,
        ficantes, casados, amantes? Urgência emocional. Uma cilada. Associamos diversas palavras ao
        AMOR: paixão, romance, sexo, adrenalina, palpitação. Esquecemos, no entanto, da palavra que
        viabiliza esse sentimento: “paciência”. Amor sem paciência não vinga. Amor não pode ser mastigado
        e engolido com emergência, com fome desesperada. É uma refeição que pode durar uma vida.
        MEDEIROS, M. Disponível em: http://porumavidasimples.blogspot.com.br. Acesso em: 20 ago. 2017
        (adaptado).   
        
        Nesse texto de opinião, as marcas linguísticas revelam uma situação distensa e de pouca formalidade, o que se evidencia pelo(a)
        
        Opções:
        A) A impessoalização ao longo do texto, com em: “se não há mais tempo”.
        B) A construção de uma atmosfera de urgência, em palavras como: “pressa”.
        C) A repetição de uma determinada estrutura sintática, como em: “Se tudo é para ontem”.
        D) O ênfase no emprego de hipérboles, como em: “uma reflexão que pode durar uma vida”.
        E) O emprego de metáforas, como em: “a vida engata uma primeira e sai em disparada”.
        
        Explicação:
        O texto é escrito em uma linguagem leve, ágil, e de pouca formalidade. Além disso, possui figuras de
        linguagem, como metáforas e hipérboles, que não são excludentes. Em uma análise sequencial das
        alternativas, daria para afirmar que D. e E. estão corretas. Entretanto, observando em detalhes, nota-se
        que a expressão "emprego de metáforas" mostra ser mais adequada do que "ênfase no emprego da
        hipérbole", visto que, para afirmarmos que o uso de hipérboles foi enfatizado, a figura de linguagem
        deveria ter aparecido mais vezes. Isso torna a alternativa E. mais provável de ser CORRETA. Além
        disso, impessoalização não deve ser apontada como marca de pouca formalidade. Existe também uma
        atmosfera de urgência, mas que é criticada no texto que destaca a importância da paciência e não da
        pressa. Por fim, a estrutura sintática não é repetida sistematicamente ao longo do texto.         
        
        Resposta final: E
        
        ---
        
        ### Exemplo 2:
        
        Pergunta:
        Sempre que a relevância do discurso entra em jogo, a questão torna-se política por
        definição, pois é o discurso que faz do homem um ser político. E tudo que os homens fazem, sabem
        ou experimentam só tem sentido na medida em que pode ser discutido. Haverá, talvez, verdades que
        ficam além da linguagem e que podem ser de grande relevância para o homem no singular, isto é, para
        o homem que, seja o que for, não é um ser político. Mas homens no plural, isto é, os homens que vivem
        e se movem e agem neste mundo, só podem experimentar o significado das coisas por poderem falar
        e ser inteligíveis entre si e consigo mesmos. ARENDT, H. A condição humana. Rio de Janeiro: Forense
        Universitária, 2004.
        
        No trecho, a filósofa Hannah Arendt mostra a importância da linguagem no processo de
        
        Opções:
        A) entendimento da cultura.
        B) aumento da criatividade.
        C) percepção da individualidade.
        D) melhoria da técnica.
        E) construção da sociabilidade.
        
        Explicação:
        Hannah Arendt defende em sua obra que somos seres políticos, no sentido próprio de vivermos
        em pólis, em ambiente coletivo e social. E essa sociabilidade só é possível por meio do discurso,
        da linguagem. Desse modo, podemos concluir que a linguagem se apresenta como uma importante
        ferramenta para a construção da sociabilidade, e portanto a alternativa E. é a CORRETA. Além disso,
        não se trata do entendimento da cultura, mas da relação social entre as pessoas dessa cultura. Hannah
        também não fala sobre aumento de criatividade, tampouco sobre técnica. Por fim, a linguagem é
        utilizada em algo mais coletivo e social, justamente o oposto da individualidade.      
        
        Resposta final: E
        
        ---
        
        ### Exemplo 3:
        
        Pergunta:
        Um casal planeja construir em sua chácara uma piscina com o formato de um paralelepípedo reto retângulo com capacidade para 90 000 L de água. O casal contratou uma empresa
        de construções que apresentou cinco projetos com diferentes combinações nas dimensões internas
        de profundidade, largura e comprimento. A piscina a ser construída terá revestimento interno em suas
        paredes e fundo com uma mesma cerâmica, e o casal irá escolher o projeto que exija a menor área de
        revestimento. As dimensões internas de profundidade, largura e comprimento, respectivamente, para
        cada um dos projetos, são: projeto I: 1,8 m, 2,0 m e 25,0 m; projeto II: 2,0 m, 5,0 m e 9,0 m; projeto III:
        1,0 m, 6,0 m e 15,0 m; projeto IV: 1,5 m, 15,0 m e 4,0 m; projeto V: 2,5 m, 3,0 m e 12,0 m.
        
        O projeto que o casal deverá escolher será o
        
        Opções:
        A) I.  
        B) II.  
        C) III.  
        D) IV.  
        E) V.
        
        Explicação:
        Devemos calcular a área das quatro faces laterais e a área da base inferior (fundo da piscina) e somar
        essas áreas para obter a área de revestimento. Logo, calculando a área de revestimento de cada
        projeto, temos: Projeto I: A = 2 x 25 + 2 x 1,8 x (2 + 25) = 147,2; Projeto II: A = 9 x 5 + 2 x 2 x (9 + 5) =
        101; Projeto III: A = 15 x 6 + 2 x 1 x (15 + 6) = 132; Projeto IV: A = 4 x 15 + 2 x 1,5 x (15 + 4) = 117;
        Projeto V: A = 3 x 12 + 2 x 2,5 x (3 + 12) = 111. Logo, o projeto com menor área de revestimento, é o
        projeto II, portanto a resposta corrreta é B.        
        
        Resposta final: B
        
        ---
        
        Agora responda à próxima pergunta seguindo o mesmo formato de raciocício passo a passo.
        """

    # Current question
    question = question_obj["question"]
    options = question_obj["alternatives"]
    option_letters = ["A", "B", "C", "D", "E"]
    formatted_options = "\n".join([f"{letter}) {text}" for letter, text in zip(option_letters, options)])

    prompt = f"""{few_shot_examples}

        Pergunta:
        {question}
        
        Opções:
        {formatted_options}
        
        Explique sua resposta e depois diga a letra da alternativa correta no formato "Resposta final: X"
        """

    return prompt


## Chain-of-Verification (CoVe) template

### Plan verifications

In [392]:
def plan_verification_questions(question, baseline_answer):
    plan_prompt = f"""Dada a seguinte pergunta e resposta, gere 2 a 4 perguntas para verificar os fatos principais da resposta.

Pergunta: {question}

Resposta: {baseline_answer}

Liste as perguntas de verificação:"""
    return call_openai_api(plan_prompt)


### Execute verifications

In [393]:
def execute_verifications(verification_questions):
    verifications = []
    for q in verification_questions:
        answer = call_openai_api(q)
        verifications.append((q, answer))
    return verifications

### Generate final verified answer

In [394]:
def generate_final_verified_answer(question, original_answer, verifications, options=None):
    vtext = "\n".join([f"Q: {q}\nA: {a}" for q, a in verifications])

    option_letters = ["A", "B", "C", "D", "E"]
    options_text = ""
    if options:
        options_text = "\n".join([f"{letter}) {text}" for letter, text in zip(option_letters, options)])

    revise_prompt = f"""
Revise a resposta abaixo com base nas verificações.

Pergunta:
{question}

Alternativas:
{options_text}

Resposta original:
{original_answer}

Verificações:
{vtext}

Resposta final verificada:
[Inclua uma explicação revisada, seguida de uma linha como: "Resposta final: X", onde X é a letra da alternativa correta.]
"""
    return call_openai_api(revise_prompt)


## Self-Refine template

### Give feedback for a previous answer

In [395]:
def build_feedback_prompt(question_obj, model_output):
    question = question_obj["question"]
    options = question_obj["alternatives"]
    option_letters = ["A", "B", "C", "D", "E"]
    formatted_options = "\n".join([f"{l}) {t}" for l, t in zip(option_letters, options)])

    prompt = (
        "Analise a seguinte resposta gerada para uma pergunta do ENEM. "
        "Identifique erros, falhas na argumentação ou escolha incorreta da alternativa. "
        "Aponte aspectos que podem ser melhorados.\n\n"
        f"Pergunta:\n{question}\n\n"
        f"Alternativas:\n{formatted_options}\n\n"
        f"Resposta do modelo:\n{model_output}\n\n"
        "Feedback:"
    )
    return prompt


### Refine response based on feedback

In [396]:
def build_refine_prompt(question_obj, model_output, feedback):
    question = question_obj["question"]
    options = question_obj["alternatives"]
    option_letters = ["A", "B", "C", "D", "E"]
    formatted_options = "\n".join([f"{l}) {t}" for l, t in zip(option_letters, options)])

    prompt = (
        "A seguir está uma pergunta do ENEM, acompanhada de alternativas, "
        "uma resposta inicial e um feedback crítico. Escreva uma nova resposta levando em conta "
        "o feedback, explicando novamente o raciocínio e indicando a letra da alternativa correta "
        "no formato \"Resposta final: X\".\n\n"
        f"Pergunta:\n{question}\n\n"
        f"Alternativas:\n{formatted_options}\n\n"
        f"Resposta anterior:\n{model_output}\n\n"
        f"Feedback:\n{feedback}\n\n"
        "Nova resposta:"
    )
    return prompt


### Feedback iteration wrapper

In [397]:
def self_refine_enem(question_obj, max_iters=10):
    # Generate the initial chain-of-thought response
    prompt = build_cot_prompt(question_obj)
    response = call_openai_api(prompt)
    
    # Extract the final answer from the initial response
    prev_final = extract_answer_letter(response)
    
    history = [(response, None)]

    for _ in range(max_iters):
        # Generate feedback based on the current response
        fb_prompt = build_feedback_prompt(question_obj, response)
        feedback = call_openai_api(fb_prompt)

        # Build the refine prompt using the current response and feedback
        refine_prompt = build_refine_prompt(question_obj, response, feedback)
        new_response = call_openai_api(refine_prompt)
        
        # Extract the final answer from the new response
        new_final = extract_answer_letter(new_response)
        
        # If the final answer is unchanged, exit the loop
        if new_final is not None and new_final == prev_final:
            break
        
        # Update the response and the final answer for the next iteration
        response = new_response
        prev_final = new_final
        history.append((response, feedback))
    
    return response, history

## Load 2024 ENEM questions

In [398]:
import json

# Load the JSONL file line by line
data = []
with open('enem_2024.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Assign subject based on the index (0-indexed)
for i, entry in enumerate(data):
    if i < 45:
        subject = "Linguagens, Códigos e suas Tecnologias"
    elif i < 90:
        subject = "Ciências Humanas e suas Tecnologias"
    elif i < 135:
        subject = "Ciências da Natureza e suas Tecnologias"
    else:
        subject = "Matemática e suas Tecnologias"
    entry["subject"] = subject

# Show the first question
data[0]

{'id': 'questao_01',
 'exam': '2024',
 'IU': False,
 'ledor': False,
 'question': '## Holy War\nOh, so we can hate each other and fear each other\nWe can build these walls between each other Baby, blow by blow and brick by brick Keep yourself locked in, yourself locked in\n[…]\nOh, maybe we should love somebody\nOh, maybe we could care a little more\nSo maybe we should love somebody\nInstead of polishing the bombs of holy war\nNessa letra de canção, de Alicia Keys, que aborda um contexto de ódio e intolerância, o marcador “instead of ” introduz a ideia de',
 'alternatives': ['mudança de comportamento.',
  'panorama de conflitos.',
  'rotina de isolamento.',
  'perspectiva bélica.',
  'cenário religioso.'],
 'label': 'A',
 'figures': [],
 'description': [],
 'subject': 'Linguagens, Códigos e suas Tecnologias'}

## Connect to OpenAI API

In [399]:
from openai import OpenAI

# Read key from file
with open("openai-key.txt", "r") as f:
    api_key = f.read().strip()

client = OpenAI(api_key=api_key)

### API call

In [400]:
def call_openai_api(prompt, model="gpt-3.5-turbo", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

## Test Prompting Techniques

### Test CoT

#### Test CoT with a single question

In [401]:
q = data[0]
prompt = build_cot_prompt(q)
response = call_openai_api(prompt)
predicted = extract_answer_letter(response)

print("Prompt:\n", prompt)
print("\nModel Response:\n", response)
print(f"\nPredicted: {predicted} | Ground Truth: {q['label']}")


Prompt:
 Você verá abaixo alguns exemplos de como a pergunta deve ser respondida passo a passo. Leia atentamente os exemplos e, em seguida, responda a pergunta que vem depois deles.

        ### Exemplo 1:

        Pergunta:
        Urgência emocional. Se tudo é para ontem, se a vida engata uma primeira e sai em
        disparada, se não há mais tempo para paradas estratégicas, caímos fatalmente no vício de querer
        que os amores sejam igualmente resolvidos num átimo de segundo. Temos pressa para ouvir “eu
        te amo”. Não vemos a hora de que fiquem estabelecidas as regras de convívio: somos namorados,
        ficantes, casados, amantes? Urgência emocional. Uma cilada. Associamos diversas palavras ao
        AMOR: paixão, romance, sexo, adrenalina, palpitação. Esquecemos, no entanto, da palavra que
        viabiliza esse sentimento: “paciência”. Amor sem paciência não vinga. Amor não pode ser mastigado
        e engolido com emergência, com fome desesperada. É uma refeição qu

#### CoT evaluation loop over sample questions

In [402]:
cot_results = []

for i, question in enumerate(data):
    prompt = build_cot_prompt(question)
    correct_answer = question["label"]
    
    try:
        response = call_openai_api(prompt)
        predicted = extract_answer_letter(response)
    except Exception as e:
        response = str(e)
        predicted = None
    
    cot_results.append({
        "id": question["id"],
        "subject": question["subject"],
        "question": question["question"],
        "ground_truth": correct_answer,
        "predicted": predicted,
        "correct": predicted == correct_answer,
        "response": response
    })

    print(f"[{i+1}/{len(data)}] ✅ Predicted: {predicted} | Correct: {correct_answer}")


[1/180] ✅ Predicted: A | Correct: A
[2/180] ✅ Predicted: D | Correct: A
[3/180] ✅ Predicted: C | Correct: C
[4/180] ✅ Predicted: E | Correct: E
[5/180] ✅ Predicted: A | Correct: A
[6/180] ✅ Predicted: C | Correct: C
[7/180] ✅ Predicted: B | Correct: B
[8/180] ✅ Predicted: E | Correct: E
[9/180] ✅ Predicted: A | Correct: D
[10/180] ✅ Predicted: A | Correct: A
[11/180] ✅ Predicted: D | Correct: D
[12/180] ✅ Predicted: E | Correct: E
[13/180] ✅ Predicted: E | Correct: E
[14/180] ✅ Predicted: B | Correct: B
[15/180] ✅ Predicted: D | Correct: D
[16/180] ✅ Predicted: D | Correct: D
[17/180] ✅ Predicted: B | Correct: B
[18/180] ✅ Predicted: D | Correct: D
[19/180] ✅ Predicted: B | Correct: B
[20/180] ✅ Predicted: B | Correct: B
[21/180] ✅ Predicted: D | Correct: C
[22/180] ✅ Predicted: D | Correct: E
[23/180] ✅ Predicted: E | Correct: E
[24/180] ✅ Predicted: B | Correct: C
[25/180] ✅ Predicted: B | Correct: B
[26/180] ✅ Predicted: C | Correct: D
[27/180] ✅ Predicted: C | Correct: C
[28/180] ✅

#### CoT Results

In [403]:
df_cot = pd.DataFrame(cot_results)
save_results_csv(df_cot, method_name="cot_few-shot")
df_cot

✅ Results saved to results/cot_few-shot/cot_few-shot_results_2025-03-28_11-16-41.csv


,id,subject,question,ground_truth,predicted,correct,response
0,questao_01,"Linguagens, Códigos e suas Tecnologias","## Holy War\nOh, so we can hate each other and fear each other\nWe can build these walls between each other Baby, blow by blow and brick by brick Keep yourself locked in, yourself locked in\n[…]\nOh, maybe we should love somebody\nOh, maybe we could care a little more\nSo maybe we should love somebody\nInstead of polishing the bombs of holy war\nNessa letra de canção, de Alicia Keys, que aborda um contexto de ódio e intolerância, o marcador “instead of ” introduz a ideia de",A,A,True,"A expressão ""instead of"" introduz a ideia de substituição ou troca, indicando que em vez de uma ação negativa, como polir as bombas da guerra santa, deveríamos realizar uma ação positiva, como amar alguém. Portanto, a ideia transmitida é de mudança de comportamento, de trocar uma atitude prejudicial por algo benéfico.\n\nResposta final: A) mudança de comportamento."
1,questao_02,"Linguagens, Códigos e suas Tecnologias",[[placeholder]]\nO texto estabelece uma relação entre elementos da natureza e comandos de um programa de computador para,A,D,False,"Para responder a essa pergunta, devemos analisar o trecho que estabelece a relação entre elementos da natureza e comandos de um programa de computador. A descrição de processos naturais, como a germinação de sementes e o crescimento de árvores, sendo comparados a comandos de um programa de computador, sugere a intenção de orientar os usuários sobre o emprego sustentável das novas tecnologias. Essa comparação pode indicar a importância de utilizar a tecnologia de forma consciente e responsável para preservar o meio ambiente.\n\nPortanto, a alternativa correta é:\n\nResposta final: D"
2,questao_03,"Linguagens, Códigos e suas Tecnologias",[[placeholder]]\nA relação entre as citações atribuídas ao físico Albert Einstein e ao cantor e compositor Bob Marley reside na crença de que é necessário,C,C,True,"Para responder a essa pergunta, precisamos analisar as citações atribuídas a Albert Einstein e Bob Marley.\n\nA citação atribuída a Albert Einstein diz: ""Insanidade é fazer sempre a mesma coisa e esperar resultados diferentes"". Essa frase sugere que é necessário mudar as ações para obter resultados diferentes.\n\nJá a citação atribuída a Bob Marley diz: ""Você nunca sabe o quão forte você é, até que ser forte é a única escolha que você tem"". Essa frase indica que é importante encontrar força em situações adversas.\n\nPortanto, a relação entre essas citações reside na ideia de que é necessário tirar proveito de situações que podem ser adversas, como sugerido pela frase de Bob Marley. Afinal, continuar fazendo sempre a mesma coisa (como na citação de Einstein) pode não trazer resultados diferentes se não houver adaptação às circunstâncias.\n\nResposta final: C"
3,questao_04,"Linguagens, Códigos e suas Tecnologias",[[placeholder]]A carta da editora Stephanie Allen-Nichols à escritoraAlice Walker tem o propósito de,E,E,True,"A carta da editora Stephanie Allen-Nichols à escritora Alice Walker tem o propósito de comunicar a recusa da publicação de seu romance. Isso pode ser deduzido pelo fato de que a editora menciona que a obra não atendeu às expectativas da editora em termos de qualidade e conteúdo. Sendo assim, a alternativa correta é a letra E.\n\nResposta final: E"
4,questao_05,"Linguagens, Códigos e suas Tecnologias","## Borderlands/La frontera: the Nem Mestiza\nGlória Anzaldúa\nI remember being caught speaking Spanish at recess [...] I remember being sent to the corner of the classroom for “talking back” to the Anglo teacher when all I was trying to do was tell her how to pronounce my name. “If you want to be American, speak ‘American’. If you don’t like it, go back to Mexico where you belong”.\n“I want you to speak English […]”, my mother would say, mortified that I spoke English like a Mexican. At Pan American University, I and all Chicano students were required to take two speech classe

### Test CoVe

#### Test CoVe with a single question

In [404]:
# Step 1: Pick a question
q = data[0]
question_text = q["question"]
prompt = build_cot_prompt(q)

# Step 2: Baseline CoT answer
baseline_answer = call_openai_api(prompt)

print("🔹 Baseline CoT Answer:\n", baseline_answer)

# Step 3: Plan verification questions
verification_qs_raw = plan_verification_questions(question_text, baseline_answer)

# Split the response into individual questions if the model gives a list
verification_questions = [line.strip("- ").strip() for line in verification_qs_raw.split("\n") if line.strip()]

print("\n🔹 Verification Questions:")
for qv in verification_questions:
    print("-", qv)

# Step 4: Execute verification
verifications = execute_verifications(verification_questions)

print("\n🔹 Verification Answers:")
for qv, av in verifications:
    print(f"Q: {qv}\nA: {av}\n")

# Step 5: Generate final verified answer
final_answer = generate_final_verified_answer(question_text, baseline_answer, verifications)

print("🔹 Final Verified Answer:\n", final_answer)

# Step 6: Extract predicted letter and compare to ground truth
predicted = extract_answer_letter(final_answer)
ground_truth = q["label"]

print(f"\n✅ Predicted Answer: {predicted}")
print(f"🎯 Ground Truth: {ground_truth}")

if predicted == ground_truth:
    print("🎉 CORRECT!")
else:
    print("❌ WRONG.")


🔹 Baseline CoT Answer:
 A expressão "instead of" é utilizada para indicar uma substituição ou troca de uma ação por outra. No contexto da letra da canção, em que se fala sobre ódio e intolerância, a ideia introduzida por "instead of polishing the bombs of holy war" é a de mudança de comportamento, sugerindo que ao invés de alimentar conflitos belicosos, deveríamos amar e cuidar uns dos outros.

Resposta final: A

🔹 Verification Questions:
- 1. Qual é o significado da expressão "instead of" na letra da canção de Alicia Keys?
- 2. Qual é a ideia introduzida pela expressão "instead of polishing the bombs of holy war" na letra da canção?
- 3. De acordo com a letra da canção, qual é a sugestão feita em relação ao comportamento humano em relação ao ódio e intolerância?
- 4. Como a expressão "instead of" contribui para o entendimento da mensagem da música em relação à guerra santa?

🔹 Verification Answers:
Q: 1. Qual é o significado da expressão "instead of" na letra da canção de Alicia Keys?

#### CoVe evaluation loop over sample questions

In [405]:
cove_results = []

for i, question in enumerate(data):
    try:
        question_text = question["question"]
        correct_answer = question["label"]

        # Build CoT-style prompt
        prompt = build_cot_prompt(question)

        # Baseline CoT response
        baseline_answer = call_openai_api(prompt)

        # Extract initial answer letter from baseline
        initial_answer_letter = extract_answer_letter(baseline_answer)

        # Plan verifications
        verification_qs_raw = plan_verification_questions(question_text, baseline_answer)
        verification_questions = [line.strip("- ").strip() for line in verification_qs_raw.split("\n") if line.strip()]

        # Execute verifications
        verifications = execute_verifications(verification_questions)

        # Final revised answer
        final_answer = generate_final_verified_answer(
            question_text, baseline_answer, verifications, question["alternatives"]
        )

        # Extract answer letter (A–E)
        predicted = extract_answer_letter(final_answer)

        # Record result
        cove_results.append({
            "id": question["id"],
            "question": question_text,
            "ground_truth": correct_answer,
            "predicted": predicted,
            "correct": predicted == correct_answer,
            "baseline_answer": baseline_answer,
            "initial_answer": initial_answer_letter,
            "final_answer": final_answer,
            "verification_qs": verification_questions,
            "verification_a": verifications,
            "subject": question.get("subject", "unknown")
        })

        print(f"[{i+1}/{len(data)}] ✅ Predicted: {predicted} | Correct: {correct_answer}")

    except Exception as e:
        print(f"[{i+1}/{len(data)}] ❌ Error: {e}")
        cove_results.append({
            "index": i,
            "question": question_text,
            "true_answer": correct_answer,
            "predicted": None,
            "correct": False,
            "error": str(e)
        })


[1/180] ✅ Predicted: A | Correct: A
[2/180] ✅ Predicted: D | Correct: A
[3/180] ✅ Predicted: C | Correct: C
[4/180] ✅ Predicted: C | Correct: E
[5/180] ✅ Predicted: A | Correct: A
[6/180] ✅ Predicted: C | Correct: C
[7/180] ✅ Predicted: B | Correct: B
[8/180] ✅ Predicted: E | Correct: E
[9/180] ✅ Predicted: A | Correct: D
[10/180] ✅ Predicted: A | Correct: A
[11/180] ✅ Predicted: D | Correct: D
[12/180] ✅ Predicted: E | Correct: E
[13/180] ✅ Predicted: E | Correct: E
[14/180] ✅ Predicted: E | Correct: B
[15/180] ✅ Predicted: D | Correct: D
[16/180] ✅ Predicted: D | Correct: D
[17/180] ✅ Predicted: B | Correct: B
[18/180] ✅ Predicted: D | Correct: D
[19/180] ✅ Predicted: B | Correct: B
[20/180] ✅ Predicted: B | Correct: B
[21/180] ✅ Predicted: D | Correct: C
[22/180] ✅ Predicted: A | Correct: E
[23/180] ✅ Predicted: E | Correct: E
[24/180] ✅ Predicted: B | Correct: C
[25/180] ✅ Predicted: B | Correct: B
[26/180] ✅ Predicted: A | Correct: D
[27/180] ✅ Predicted: C | Correct: C
[28/180] ✅

#### CoVe results

In [406]:
df_cove = pd.DataFrame(cove_results)
save_results_csv(df_cove, method_name="cove_few-shot")
df_cove

✅ Results saved to results/cove_few-shot/cove_few-shot_results_2025-03-28_12-04-04.csv


,id,question,ground_truth,predicted,correct,baseline_answer,initial_answer,final_answer,verification_qs,verification_a,subject
0,questao_01,"## Holy War\nOh, so we can hate each other and fear each other\nWe can build these walls between each other Baby, blow by blow and brick by brick Keep yourself locked in, yourself locked in\n[…]\nOh, maybe we should love somebody\nOh, maybe we could care a little more\nSo maybe we should love somebody\nInstead of polishing the bombs of holy war\nNessa letra de canção, de Alicia Keys, que aborda um contexto de ódio e intolerância, o marcador “instead of ” introduz a ideia de",A,A,True,"A expressão ""instead of"" indica substituição, ou seja, sugere que é possível fazer algo diferente do que está sendo feito atualmente. No contexto da letra da música, em que se fala sobre ódio, intolerância e guerra santa, a ideia introduzida por ""instead of polishing the bombs of holy war"" é a de mudança de comportamento, de atitudes. A sugestão é amar alguém em vez de se envolver em conflitos e guerras.\n\nResposta final: A) mudança de comportamento.",A,"A expressão ""instead of"" sugere substituição ou troca de algo por outra coisa. Na letra da música, ela introduz a ideia de mudança de comportamento, de atitudes, incentivando a escolha de amar e cuidar ao invés de se envolver em conflitos e guerras. Resposta final: A) mudança de comportamento.","[1) Qual é a expressão que indica substituição na letra da música de Alicia Keys?, 2) O que a expressão ""instead of polishing the bombs of holy war"" sugere fazer em vez de se envolver em conflitos e guerras?, 3) Qual é a ideia introduzida pela expressão ""instead of"" na letra da música?, 4) Como a expressão ""instead of"" está relacionada à mensagem de amor e tolerância na música de Alicia Keys?]","[(1) Qual é a expressão que indica substituição na letra da música de Alicia Keys?, A expressão é ""cover"".), (2) O que a expressão ""instead of polishing the bombs of holy war"" sugere fazer em vez de se envolver em conflitos e guerras?, A expressão sugere investir tempo e esforço em atividades pacíficas e construtivas, em vez de se envolver em conflitos e guerras. Isso pode incluir buscar soluções diplomáticas, promover o diálogo e a compreensão mútua, e trabalhar para construir um mundo mais pacífico e harmonioso. Em vez de preparar e alimentar a violência, a sugestão é focar em promover a paz e a cooperação entre os povos.), (3) Qual é a ideia introduzida pela expressão ""instead of"" na letra da música?, A expressão ""instead of"" introduz a ideia de substituição ou troca de algo por outra coisa. Na letra da música, provavelmente está sendo sugerido que em vez de fazer algo de uma maneira específica, seria melhor fazer de outra forma. Pode estar indicando uma preferência por uma alternativa diferente.), (4) Como a expressão ""instead of"" está relacionada à mensagem de amor e tolerância na música de Alicia Keys?, A expressão ""instead of"" está relacionada à mensagem de amor e tolerância na música de Alicia Keys ao sugerir uma escolha positiva e construtiva em vez de uma negativa ou destrutiva. Ao invés de responder ao ódio com mais ódio, a cantora incentiva a responder com amor e compreensão. Isso reflete a mensagem central de amor e tolerância que Alicia Keys transmite em suas músicas, promovendo a importância de escolhas conscientes e positivas para promover a paz e a harmonia entre as pessoas.)]","Linguagens, Códigos e suas Tecnologias"
1,questao_02,[[placeholder]]\nO texto estabelece uma relação entre elementos da natureza e comandos de um programa de computador para,A,D,False,"Para responder a essa pergunta, precisamos analisar o trecho em que o texto estabelece a relação entre elementos da natureza e comandos de um programa de computador. A comparação entre o crescimento de uma árvore e o carregamento de um vídeo em uma tela sugere uma analogia entre a natureza e a tecnologia. Nesse sentido, a opção que mais se adequa a essa relação é a alternativa D

### Test Self-Refine

#### Test Self-Refine with a single question 

In [407]:
q = data[0]
final_response, trace = self_refine_enem(q)
predicted = extract_answer_letter(final_response)

print("Prompt inicial (iteração 0):\n", build_cot_prompt(q))
print("\nResposta final após refinamento:\n", final_response)
print(f"\nAlternativa prevista: {predicted} | Gabarito: {q['label']}")

print("\nHistórico de iterações:")
for i, (resp, fb) in enumerate(trace):
    print(f"\n--- Iteração {i} ---")
    print("Resposta:", resp)
    if fb:
        print("Feedback:", fb)


Prompt inicial (iteração 0):
 Você verá abaixo alguns exemplos de como a pergunta deve ser respondida passo a passo. Leia atentamente os exemplos e, em seguida, responda a pergunta que vem depois deles.

        ### Exemplo 1:

        Pergunta:
        Urgência emocional. Se tudo é para ontem, se a vida engata uma primeira e sai em
        disparada, se não há mais tempo para paradas estratégicas, caímos fatalmente no vício de querer
        que os amores sejam igualmente resolvidos num átimo de segundo. Temos pressa para ouvir “eu
        te amo”. Não vemos a hora de que fiquem estabelecidas as regras de convívio: somos namorados,
        ficantes, casados, amantes? Urgência emocional. Uma cilada. Associamos diversas palavras ao
        AMOR: paixão, romance, sexo, adrenalina, palpitação. Esquecemos, no entanto, da palavra que
        viabiliza esse sentimento: “paciência”. Amor sem paciência não vinga. Amor não pode ser mastigado
        e engolido com emergência, com fome desespera

#### Self-Refine evaluation loop over sample questions

In [408]:
self_refine_results = []

for i, question in enumerate(data):
    try:
        question_text = question["question"]
        correct_answer = question["label"]

        # 1. Run SELF-REFINE
        final_answer, trace = self_refine_enem(question, max_iters=3)
        predicted = extract_answer_letter(final_answer)

        # 2. Extract intermediate answers from trace
        answer_sequence = []
        baseline_answer = None
        
        for step, (response, feedback) in enumerate(trace):
            try:
                letter = extract_answer_letter(response)
            except Exception:
                letter = None
            answer_sequence.append(letter)
            if step == 0:
                baseline_answer = response

        # 3. Record result
        self_refine_results.append({
            "id": question["id"],
            "question": question_text,
            "ground_truth": correct_answer,
            "predicted": predicted,
            "correct": predicted == correct_answer,
            "baseline_answer": baseline_answer,
            "final_answer": final_answer,
            "answer_sequence": answer_sequence, 
            "trace": trace,
            "subject": question.get("subject", "unknown")
        })

        print(f"[{i+1}/{len(data)}] ✅ Predicted: {predicted} | Correct: {correct_answer}")

    except Exception as e:
        print(f"[{i+1}/{len(data)}] ❌ Error: {e}")
        self_refine_results.append({
            "index": i,
            "question": question.get("question", ""),
            "true_answer": question.get("label", ""),
            "predicted": None,
            "correct": False,
            "error": str(e),
            "subject": question.get("subject", "unknown")
        })


[1/180] ✅ Predicted: A | Correct: A
[2/180] ✅ Predicted: A | Correct: A
[3/180] ✅ Predicted: C | Correct: C
[4/180] ✅ Predicted: E | Correct: E
[5/180] ✅ Predicted: A | Correct: A
[6/180] ✅ Predicted: E | Correct: C
[7/180] ✅ Predicted: B | Correct: B
[8/180] ✅ Predicted: E | Correct: E
[9/180] ✅ Predicted: A | Correct: D
[10/180] ✅ Predicted: A | Correct: A
[11/180] ✅ Predicted: D | Correct: D
[12/180] ✅ Predicted: E | Correct: E
[13/180] ✅ Predicted: E | Correct: E
[14/180] ✅ Predicted: B | Correct: B
[15/180] ✅ Predicted: D | Correct: D
[16/180] ✅ Predicted: D | Correct: D
[17/180] ✅ Predicted: B | Correct: B
[18/180] ✅ Predicted: D | Correct: D
[19/180] ✅ Predicted: A | Correct: B
[20/180] ✅ Predicted: B | Correct: B
[21/180] ✅ Predicted: D | Correct: C
[22/180] ✅ Predicted: E | Correct: E
[23/180] ✅ Predicted: E | Correct: E
[24/180] ✅ Predicted: B | Correct: C
[25/180] ✅ Predicted: E | Correct: B
[26/180] ✅ Predicted: C | Correct: D
[27/180] ✅ Predicted: C | Correct: C
[28/180] ✅

#### Self-Refine Results

In [409]:
df_self_refine = pd.DataFrame(self_refine_results)
save_results_csv(df_self_refine, method_name="self-refine_few-shot")
df_self_refine

✅ Results saved to results/self-refine_few-shot/self-refine_few-shot_results_2025-03-28_12-33-28.csv


,id,question,ground_truth,predicted,correct,baseline_answer,final_answer,answer_sequence,trace,subject
0,questao_01,"## Holy War\nOh, so we can hate each other and fear each other\nWe can build these walls between each other Baby, blow by blow and brick by brick Keep yourself locked in, yourself locked in\n[…]\nOh, maybe we should love somebody\nOh, maybe we could care a little more\nSo maybe we should love somebody\nInstead of polishing the bombs of holy war\nNessa letra de canção, de Alicia Keys, que aborda um contexto de ódio e intolerância, o marcador “instead of ” introduz a ideia de",A,A,True,"A expressão ""instead of"" é utilizada para indicar uma substituição, uma troca por algo diferente. No contexto da música, a frase ""Instead of polishing the bombs of holy war"" sugere que, em vez de investir em armamentos e guerras sagradas, deveríamos amar alguém e nos importar mais com o próximo. Portanto, o marcador ""instead of"" introduz a ideia de mudança de comportamento, de substituição da violência pelo amor e pela compaixão.\n\nResposta final: A","A expressão ""instead of"" é utilizada para indicar uma substituição, uma troca por algo diferente. No contexto da música, a frase ""Instead of polishing the bombs of holy war"" sugere que, em vez de investir em armamentos e guerras sagradas, deveríamos amar alguém e nos importar mais com o próximo. Portanto, o marcador ""instead of"" introduz a ideia de mudança de comportamento, de substituição da violência pelo amor e pela compaixão.\n\nResposta final: A",[A],"[(A expressão ""instead of"" é utilizada para indicar uma substituição, uma troca por algo diferente. No contexto da música, a frase ""Instead of polishing the bombs of holy war"" sugere que, em vez de investir em armamentos e guerras sagradas, deveríamos amar alguém e nos importar mais com o próximo. Portanto, o marcador ""instead of"" introduz a ideia de mudança de comportamento, de substituição da violência pelo amor e pela compaixão.\n\nResposta final: A, None)]","Linguagens, Códigos e suas Tecnologias"
1,questao_02,[[placeholder]]\nO texto estabelece uma relação entre elementos da natureza e comandos de um programa de computador para,A,A,True,"Para responder a essa pergunta, primeiro precisamos identificar no texto a relação entre elementos da natureza e comandos de um programa de computador. No trecho apresentado, o texto compara a natureza a um computador programado para seguir processos de crescimento e desenvolvimento de forma rápida e acelerada. Essa comparação sugere uma relação entre a natureza e a tecnologia, destacando como a natureza pode ser vista como um sistema complexo e dinâmico, assim como um programa de computador.\n\nDessa forma, a opção correta que melhor representa essa relação é a alternativa D) orientar os usuários sobre o emprego sustentável das novas tecnologias.\n\nResposta final: D","Ao analisar o texto, percebemos que a relação estabelecida entre elementos da natureza e comandos de um programa de computador não se trata apenas de uma comparação entre a natureza e a tecnologia, mas sim de uma reflexão sobre a rápida destruição da natureza e a forma como ela segue processos de crescimento e desenvolvimento. A analogia com um programa de computador programado para crescer rapidamente destaca a urgência em conscientizar as pessoas sobre a importância de um emprego sustentável das novas tecnologias para preservar o meio ambiente.\n\nDessa forma, a opção correta que melhor representa essa relação é a alternativa A) alertar as pessoas sobre a rápida destruição da natureza.\n\nResposta final: A","[D, A]","[(Para responder a essa pergunta, primeiro precisamos identificar no texto a relação entre elementos da natureza e comandos de um programa de computador. No trecho apresentado, o texto compara a natureza a um computador programado para seguir processos de crescimento e desenvolvimento de forma rápida e acelerada. Essa comparação sugere uma relação entre a natureza e a tecnologia, destacando como